In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Importing all necessary libraries
import pandas as pd, numpy as np
import tensorflow as tf
assert tf.__version__ >= '2.0'

from itertools import islice

# Keras
from keras.layers import Dense, Embedding, LSTM, Dropout, MaxPooling1D, Conv1D
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.preprocessing import sequence
from keras.datasets import imdb

from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Suppress warnings
import warnings; warnings.filterwarnings('ignore')

random_state = 42
np.random.seed(random_state)
tf.random.set_seed(random_state)

### Dataset

In [7]:
imdb.load_data() 

((array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
         list([1, 19

 **Import the data**
* Use imdb.load_data() method
* Get train and test set
* Take 10000 most frequent words

In [5]:
vocab_size = 10000
maxlen = 300
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = vocab_size)

 #Pad each sentence to be of same length 
x_train = pad_sequences(x_train, maxlen = maxlen, padding = 'post')
x_test =  pad_sequences(x_test, maxlen = maxlen, padding = 'post')

X = np.concatenate((x_train, x_test), axis = 0)
y = np.concatenate((y_train, y_test), axis = 0)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle = True)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42, shuffle = True)

Print shape of features & labels 

In [9]:
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((32000, 300), (8000, 300), (32000,), (8000,))

### Decode the feature value to get original sentence

In [10]:

def decode_review(x, y):
  w2i = imdb.get_word_index()                                
  w2i = {k:(v + 3) for k, v in w2i.items()}
  w2i['<PAD>'] = 0
  w2i['<START>'] = 1
  w2i['<UNK>'] = 2
  i2w = {i: w for w, i in w2i.items()}

  ws = (' '.join(i2w[i] for i in x))
  print(f'Review: {ws}')
  print(f'Actual Sentiment: {y}')
  return w2i, i2w

w2i, i2w = decode_review(x_train[0], y_train[0])

# get first 50 key, value pairs from id to word dictionary
print('---'*30, '\n', list(islice(i2w.items(), 0, 50)))

Review: <START> the only possible way to enjoy this flick is to bang your head against the wall allow some internal <UNK> of the brain let a bunch of your brain cells die and once you are officially mentally retarded perhaps then you might enjoy this film br br the only saving grace was the story between <UNK> and stephanie govinda was excellent in the role of the cab driver and so was the brit girl perhaps if they would have created the whole movie on their <UNK> in india and how they eventually fall in love would have made it a much more enjoyable film br br the only reason i gave it a 3 rating is because of <UNK> and his ability as an actor when it comes to comedy br br <UNK> <UNK> and anil kapoor were wasted needlessly plus the scene at <UNK> of the re union was just too much to <UNK> being an international <UNK> in the post 9 11 world anil kapoor would have got himself shot much before he even reached the sky bridge to <UNK> his true love but then again the point of the movie was 

### Define model
### Compile the model
###Print model summary

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 256, input_length = maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(256, 5, padding = 'same', activation = 'relu', strides = 1))
model.add(Conv1D(128, 5, padding = 'same', activation = 'relu', strides = 1))
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(64, 5, padding = 'same', activation = 'relu', strides = 1))
model.add(MaxPooling1D(pool_size = 2))
model.add(LSTM(75))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

# Adding callbacks
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 0)  
mc = ModelCheckpoint('imdb_model.h5', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 256)          2560000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 256)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 300, 256)          327936    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 300, 128)          163968    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 150, 128)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 150, 64)           41024     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 75, 64)           

###Fit the model
### Evaluate the model

In [14]:
# Fit the model
model.fit(x_train, y_train, validation_data = (x_valid, y_valid), epochs = 3, batch_size = 64, verbose = True, callbacks = [es, mc])

# Evaluate the model
scores = model.evaluate(x_test, y_test, batch_size = 64)
print('Test accuracy: %.2f%%' % (scores[1]*100))

Epoch 1/3
500/500 [==============================] - ETA: 0s - loss: 0.5875 - accuracy: 0.6459
Epoch 00001: val_loss improved from inf to 0.37443, saving model to imdb_model.h5
500/500 [==============================] - 708s 1s/step - loss: 0.5875 - accuracy: 0.6459 - val_loss: 0.3744 - val_accuracy: 0.8359
Epoch 2/3
500/500 [==============================] - ETA: 0s - loss: 0.2753 - accuracy: 0.8869
Epoch 00002: val_loss improved from 0.37443 to 0.26486, saving model to imdb_model.h5
500/500 [==============================] - 705s 1s/step - loss: 0.2753 - accuracy: 0.8869 - val_loss: 0.2649 - val_accuracy: 0.8901
Epoch 3/3
500/500 [==============================] - ETA: 0s - loss: 0.1867 - accuracy: 0.9292
Epoch 00003: val_loss improved from 0.26486 to 0.26232, saving model to imdb_model.h5
157/157 [==============================] - 50s 321ms/step - loss: 0.2546 - accuracy: 0.8954
Test accuracy: 89.54%


###Predict on one sample 

In [15]:
y_pred = model.predict_classes(x_test)
print(f'Classification Report:\n{classification_report(y_pred, y_test)}')

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.91      0.89      4668
           1       0.92      0.88      0.90      5332

    accuracy                           0.90     10000
   macro avg       0.89      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



Retrive output of each layer in keras for a given single test sample from the trained model

In [18]:
sample_x_test = x_test[np.random.randint(10000)]
for layer in model.layers:

    model_layer = Model(inputs = model.input, outputs = model.get_layer(layer.name).output)
    output = model_layer.predict(sample_x_test.reshape(1,-1))
    print('\n','--'*20, layer.name, 'layer', '--'*20, '\n')
    print(output)


 ---------------------------------------- embedding_1 layer ---------------------------------------- 

[[[ 0.01937116  0.01736815 -0.07658639 ... -0.01544633 -0.00889149
   -0.05324595]
  [-0.03336601  0.0034871  -0.00826419 ...  0.02852849 -0.00762371
   -0.0515163 ]
  [-0.13579968 -0.0724849   0.07188319 ...  0.06465711  0.04716221
   -0.04557375]
  ...
  [-0.04701929 -0.03804651  0.03794101 ...  0.04789864  0.00673577
    0.03321566]
  [-0.04701929 -0.03804651  0.03794101 ...  0.04789864  0.00673577
    0.03321566]
  [-0.04701929 -0.03804651  0.03794101 ...  0.04789864  0.00673577
    0.03321566]]]

 ---------------------------------------- dropout_1 layer ---------------------------------------- 

[[[ 0.01937116  0.01736815 -0.07658639 ... -0.01544633 -0.00889149
   -0.05324595]
  [-0.03336601  0.0034871  -0.00826419 ...  0.02852849 -0.00762371
   -0.0515163 ]
  [-0.13579968 -0.0724849   0.07188319 ...  0.06465711  0.04716221
   -0.04557375]
  ...
  [-0.04701929 -0.03804651  0.037

In [19]:
decode_review(x_test[10], y_test[10])
print(f'Predicted sentiment: {y_pred[10][0]}')

Review: <START> this movie was great and i was waiting for it for a long time when it finally came out i was really happy and looked forward to a 10 out of 10 it was great and lived up to my potential the performances were great on the part of the adults and most of the kids the only bad performance was by milo himself there was one problem that i encountered with this and others like it movie all of the characters i wanted to live were getting killed overall i give this movie an excellent 9 out of 10 maybe we should <UNK> better people to kill next time though ok <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PA

Conclusion :
Sentiment classification task on  test dataset is achieved 
Accuracy: 92% and f1-score: 90%
